In [1]:
import pandas as pd
import xgboost as xgb
import os

import dtale

import scipy.stats as stats

import numpy as np

import psycopg2

####### read files: #########
def readDF(fileDir):
    featureDF = pd.read_csv(fileDir)
    return featureDF

### descrição do problema:

### Ja ta com as features
### Ja ta tudo na mesma tabela
### Problema de classificacao

###

filePathData = os.path.join(os.getcwd(),"data")
filePathSQL = os.path.join(os.getcwd(),"data","tmp")

fileDir = os.path.join(filePathData, "tabular-playground-series-may-2022", "train.csv")

featureDF = readDF(fileDir)
idxName = ['id']

featureDF.set_index(idxName[0],inplace=True,drop=False)

pandasSQLTypes={"int64":"bigint","object":"varchar(100)","float64":"double precision"}

#catFList = ['f_27']
catFList = ['f_27',"f_07","f_08","f_09","f_10","f_11","f_12"]
#discFList = ["f_07","f_08","f_09","f_10","f_11","f_12","f_13","f_14","f_15","f_16","f_17","f_18","f_29","f_30"]
discFList = ["f_13","f_14","f_15","f_16","f_17","f_18","f_29","f_30"]
tgtFList = ["target"]
numFList = list(set(featureDF.columns) - set(tgtFList) - set(discFList) - set(catFList))
numFList.sort()

numDF = featureDF
numDF = numDF.sample(n=50000)

numDFMelt = pd.melt(numDF, id_vars = idxName + catFList, value_vars=numFList+discFList,value_name='numFeatures')

dim_fDF = pd.DataFrame(list(featureDF.columns),columns=["variable"])
dim_fDF.drop_duplicates(inplace=True)
dim_idDF = pd.DataFrame(list(featureDF.index),columns=idxName)
dim_idDF.drop_duplicates(inplace=True)

In [2]:
numDFMelt.head(5)

,id,f_27,f_07,f_08,f_09,f_10,f_11,f_12,variable,numFeatures
0,454224,BFBAAABRCA,5,3,2,0,0,2,f_00,-0.825318
1,270530,AFBCCABDFB,0,0,4,3,1,3,f_00,-0.463939
2,166019,ACBBCABSCD,3,3,2,2,2,0,f_00,-0.861289
3,161152,AEBBCBCDED,0,0,3,4,4,3,f_00,0.660846
4,232649,ABBBCAGFAE,1,0,2,3,3,5,f_00,0.694640


In [3]:
colunas = list(numDFMelt['variable'].drop_duplicates())

dfList = []
binListNp = []
binListTmp = []
binList = []
for col in colunas:

    numDFMelt_1 = (numDFMelt[numDFMelt['variable'] == col]).copy()
    #bins = pd.cut(numDFMelt_1['numFeatures'], 3)

    binQuant = int(round((len(numDFMelt_1))**(1/2),0))

    binList = pd.cut(numDFMelt_1['numFeatures'], binQuant)

    #nao funciona para as slices?

    numDFMelt_1['ranking'] = (numDFMelt_1['numFeatures'].rank(axis=0,method='max',ascending=True)-0.05) / len(numDFMelt_1)
    numDFMelt_1['qqplot'] = stats.norm.ppf(numDFMelt_1['ranking'])

    for rows in binList:
        binListTmp.append([col,rows.left])

    binListNp = np.array(binListTmp)
    binListNp = np.append(binListNp, np.array([numDFMelt_1['id'].to_numpy()]).T, axis=1)
    binListNp = np.append(binListNp, np.array([numDFMelt_1['qqplot'].to_numpy()]).T, axis=1)

    dfList.append(pd.DataFrame(binListNp,columns=['variable','bin','id','qqplot']))

    binListTmp = []
    binListNp = []

binDf = pd.concat(dfList)
binDf['qqplot'] = binDf['qqplot'].astype('float64')
binDf['bin'] = binDf['bin'].astype('float64')
binDf['id'] = binDf['id'].astype('int64')

##slow
numDFMelt = numDFMelt.merge(binDf, how='left',on=["id","variable"])

In [4]:
numDFx = numDF.drop(columns=['id'])
d = dtale.show(numDFx)
print(d._url)
d.open_browser()

http://deskCasa:40000


In [5]:
def createCorrelDFrame(df):

    correl = df.corr()
    correl = pd.melt(correl, value_vars=numFList+discFList+tgtFList,value_name='correl', ignore_index=False)
    correl.reset_index(inplace=True)
    correl = correl.rename(columns={'index':'variable1','variable':'variable2'})
    
    return correl

def createColumnStr(df,typeMap=pandasSQLTypes):

    dfcolumns = df.columns
    dfTypes = df.dtypes

    strList = ""

    for idx in range(0,len(dfcolumns)):

        if idx == len(dfcolumns)-1 and idx == 0:
            strList = "(" + strList + " " + str(dfcolumns[idx]) + " " + typeMap[str(dfTypes[idx])] + ")"
        elif idx == len(dfcolumns)-1:
            strList = strList + " " + str(dfcolumns[idx]) + " " + typeMap[str(dfTypes[idx])] + ")"
        elif idx == 0:
            strList = "(" + str(dfcolumns[idx]) + " " + typeMap[str(dfTypes[idx])] + ","
        else:
            strList = strList + " " + str(dfcolumns[idx]) + " " + typeMap[str(dfTypes[idx])] + ","

    return strList

############
def copyDataToSQL(df,tablename,path = filePathSQL):

    pg_conn  = psycopg2.connect(database="postgres", user='postgres', password='', host='127.0.0.1', port='5432')
    cur = pg_conn.cursor()

    filePath = os.path.join(path,tablename + ".csv")

    df.to_csv(filePath,index=False, header=False)

    strList = createColumnStr(df)

    sqlCreate = '''CREATE TABLE IF NOT EXISTS ''' + tablename + ''' ''' + strList
    print(sqlCreate)

    sqlCopy = '''COPY ''' + tablename + ''' FROM ''' + "'"+filePath+"'" + ''' DELIMITER ',' CSV; '''
    print(sqlCopy)

    ### criando tabela de apoio para o direct query do BI
    cur.execute("CREATE TABLE IF NOT EXISTS _measures (_measures int); TRUNCATE TABLE _measures; INSERT INTO _measures (_measures) VALUES (1);")
    ### tem q criar a tabela de data

    cur.execute(sqlCreate)
    cur.execute('TRUNCATE TABLE ' + tablename)
    cur.execute(sqlCopy)
    pg_conn.commit()
    cur.close()

    #sql_delete = '''DROP TABLE IF EXISTS copy_test;'''

    os.remove(os.path.join(path,tablename + ".csv"))

    return 1


copyDataToSQL(createCorrelDFrame(numDF[numFList+tgtFList+discFList]),"correlTable")
copyDataToSQL(numDF,"Numeric")
copyDataToSQL(numDFMelt,"meltedNumeric")
copyDataToSQL(dim_fDF,"dim_Features")
copyDataToSQL(dim_idDF,"dim_id")

CREATE TABLE IF NOT EXISTS correlTable (variable1 varchar(100), variable2 varchar(100), correl double precision)
COPY correlTable FROM 'd:\Projetos\competicoes\featureOps\data\tmp\correlTable.csv' DELIMITER ',' CSV; 
CREATE TABLE IF NOT EXISTS Numeric (id bigint, f_00 double precision, f_01 double precision, f_02 double precision, f_03 double precision, f_04 double precision, f_05 double precision, f_06 double precision, f_07 bigint, f_08 bigint, f_09 bigint, f_10 bigint, f_11 bigint, f_12 bigint, f_13 bigint, f_14 bigint, f_15 bigint, f_16 bigint, f_17 bigint, f_18 bigint, f_19 double precision, f_20 double precision, f_21 double precision, f_22 double precision, f_23 double precision, f_24 double precision, f_25 double precision, f_26 double precision, f_27 varchar(100), f_28 double precision, f_29 bigint, f_30 bigint, target bigint)
COPY Numeric FROM 'd:\Projetos\competicoes\featureOps\data\tmp\Numeric.csv' DELIMITER ',' CSV; 
CREATE TABLE IF NOT EXISTS meltedNumeric (id bigint, f_2

1

Exception occurred while processing request: 'NoneType' object has no attribute 'columns'
Traceback (most recent call last):
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\views.py", line 2707, in get_column_analysis
    analysis = ColumnAnalysis(data_id, request)
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\column_analysis.py", line 87, in __init__
    self.selected_col = find_selected_column(
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\utils.py", line 664, in find_selected_column
    return col if col in data.columns else data.columns[-1]
AttributeError: 'NoneType' object has no attribute 'columns'


2022-06-20 10:27:13,562 - ERROR    - Exception occurred while processing request: 'NoneType' object has no attribute 'columns'
Traceback (most recent call last):
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\views.py", line 2707, in get_column_analysis
    analysis = ColumnAnalysis(data_id, request)
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\column_analysis.py", line 87, in __init__
    self.selected_col = find_selected_column(
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\utils.py", line 664, in find_selected_column
    return col if col in data.columns else data.columns[-1]
AttributeError: 'NoneType' object has no attribute 'columns'


Exception occurred while processing request: 'NoneType' object has no attribute 'columns'
Traceback (most recent call last):
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\views.py", line 120, in _handle_exceptions
    return func(*args, **kwargs)
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\views.py", line 2707, in get_column_analysis
    analysis = ColumnAnalysis(data_id, request)
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\column_analysis.py", line 87, in __init__
    self.selected_col = find_selected_column(
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\utils.py", line 664, in find_selected_column
    return col if col in data.columns else data.columns[-1]
AttributeError: 'NoneType' object has no attribute 'columns'


...

2022-06-20 11:27:18,028 - INFO     - Executing shutdown due to inactivity...


Executing shutdown...


2022-06-20 11:27:26,231 - INFO     - Executing shutdown...


Exception on /shutdown [GET]
Traceback (most recent call last):
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\app.py", line 410, in shutdown
    shutdown_server()
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\app.py", line 396, in shutdown_server
    raise RuntimeError("Not running with the Wer

2022-06-20 11:27:26,233 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\app.py", line 410, in shutdown
    shutdown_server()
  File "c:\Users\almei\.conda\envs\tabularKaggle\lib\site-packages\dtale\app.py", line 396, in shutdown_server
    raise R